# Topic distribution for PoInf / Publications

Using the bag of words generated from the corpus, and the topic models generated, I now use them to project publications in the LDA topic space, as well as for the individuals in the School

In [1]:
import pandas as pd
import cPickle as pkl

In [12]:
from gensim import models
from gensim.corpora import Dictionary

In [6]:
lookup_pub = pkl.load(open('../infnet-analysis/data/lookup_pub.pkl', 'rb'))
lookup_poinf = pkl.load(open('../infnet-analysis/data/lookup_poinf.pkl','rb'))
pub_toks = pkl.load(open('../infnet-scrapper/data/pub_toks.pkl','rb'))

In [4]:
lookup_pub.head(4)

,year,authors,shortnames,collab_id,edges
pub_id,,,,,
400818dc-63af-4a26-80c5-906f98e1f8ab,1989,d.p. o'brien|r. galvao|m. keilhacker|e. lazzar...,"brien, d. p.|galvao, r.|keilhacker, m.|lazzaro...","[324, 1942, 560, 5635, 5099, a78a09b5-138e-407...","[(324, 1942), (324, 560), (324, 5635), (324, 5..."
18b1a861-afef-4fff-bc80-d02e05be18c4,2013,paolo guagliardo|piotr wieczorek,"guagliardo, p.|wieczorek, p.","[653c9723-b374-4ad3-9cef-0f7e7f45d812, 4953]","[(653c9723-b374-4ad3-9cef-0f7e7f45d812, 4953)]"
309fdbfc-227b-4588-9264-f0f4e3cadfcb,1994,p. buneman|l. libkin|d. suciu|v. tannen|l. wong,"buneman, p.|libkin, l.|suciu, d.|tannen, v. |w...","[90396347-473b-4fba-8b1d-a1d25ea18b6b, a22b33f...","[(90396347-473b-4fba-8b1d-a1d25ea18b6b, a22b33..."
d5814bab-5fc2-4c31-92b7-543c7ce75cb4,2012,p.l. de leon|m. pucher|j. yamagishi|i. hernaez...,"leon, p. l.|pucher, m.|yamagishi, j.|hernaez, ...","[1180, 5318, 4d11a99a-0d04-42f4-8089-d433a57c2...","[(1180, 5318), (1180, 4d11a99a-0d04-42f4-8089-..."


In [5]:
lookup_poinf.head(4)

,last_name,first_name,perseonal_url,position,parent,institute,full_name,institute_class,alias
id,,,,,,,,,
003ec9bb-18aa-4e6e-95e9-359f0968262a,gray,gavin,http://www.research.ed.ac.uk/portal/en/persons...,research assistant,school of informatics,institute for computing systems architecture,gray gavin,5,NaN
010f9bf0-c04c-4cfb-ab3d-ca150de1e706,jackson,paul,http://www.research.ed.ac.uk/portal/en/persons...,senior lecturer,school of informatics,institute for computing systems architecture,jackson paul,5,"jackson, p. b.|jackson, p."
02c86de2-0fc9-4f6d-aee9-93b0f7557c84,franke,bjoern,http://www.research.ed.ac.uk/portal/en/persons...,reader,school of informatics,institute of language cognition and computation,franke bjoern,2,"franke, b."
02df029e-077b-407c-8e24-1392a9ed8060,sevilla-lara,laura,http://www.research.ed.ac.uk/portal/en/persons...,lecturer reader in image and vision computing,school of informatics,UNKNOWN,sevilla-lara laura,0,NaN


In [7]:
pub_toks.head(4)

,year,summary_toks
pub_id,,
400818dc-63af-4a26-80c5-906f98e1f8ab,1989,"[balloon, stabil, analysi, jet, hmode, dischar..."
18b1a861-afef-4fff-bc80-d02e05be18c4,2013,"[queri, process, data, integr, chapter, illust..."
309fdbfc-227b-4588-9264-f0f4e3cadfcb,1994,"[comprehens, syntax, syntax, comprehens, close..."
d5814bab-5fc2-4c31-92b7-543c7ce75cb4,2012,"[evalu, speaker, verif, secur, detect, hmmbase..."


# Topic Distribution by Publications

In [8]:
# Load Dictionary to convert words to id:
dictionary = pkl.load(open('../topicModel/dictionary.pkl','rb'))

In [9]:
# Convert tokens to bow:
bowified = lambda row: dictionary.doc2bow(row.summary_toks)
pub_toks['bow'] = pub_toks.apply(bowified, axis=1)

In [11]:
pub_toks.head(4)

,year,summary_toks,bow
pub_id,,,
400818dc-63af-4a26-80c5-906f98e1f8ab,1989,"[balloon, stabil, analysi, jet, hmode, dischar...","[(40, 2), (156, 1), (223, 2), (293, 2), (328, ..."
18b1a861-afef-4fff-bc80-d02e05be18c4,2013,"[queri, process, data, integr, chapter, illust...","[(40, 1), (229, 1), (253, 1), (319, 2), (330, ..."
309fdbfc-227b-4588-9264-f0f4e3cadfcb,1994,"[comprehens, syntax, syntax, comprehens, close...","[(90, 3), (123, 1), (186, 1), (189, 5), (196, ..."
d5814bab-5fc2-4c31-92b7-543c7ce75cb4,2012,"[evalu, speaker, verif, secur, detect, hmmbase...","[(18, 1), (28, 1), (29, 2), (30, 1), (63, 1), ..."


In [13]:
# load the LDA models:
fullpubLDA = models.LdaModel.load('fullpub.ldamodel')

In [14]:
ldaVect = pub_toks['bow'][0]

In [28]:
[0]*10

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [36]:
def inference(ldaModel, ldaVector):
    num_topics = ldaModel.num_topics
    topic_dist = ldaModel[ldaVector]
    
    # index the topic_distribution according to the distribution:
    out = [0]*num_topics
    for (i,v) in topic_dist:
        out[i] = v
    assert len(out) == num_topics
    return out

In [37]:
_inference = lambda row: inference(fullpubLDA, row.bow)
pub_toks['topic_distribution'] = pub_toks.apply(_inference, axis=1)

In [38]:
pub_toks.head(4)

,year,summary_toks,bow,topic_distribution
pub_id,,,,
400818dc-63af-4a26-80c5-906f98e1f8ab,1989,"[balloon, stabil, analysi, jet, hmode, dischar...","[(40, 2), (156, 1), (223, 2), (293, 2), (328, ...","[0, 0.0982817732416, 0, 0.0248424111021, 0.422..."
18b1a861-afef-4fff-bc80-d02e05be18c4,2013,"[queri, process, data, integr, chapter, illust...","[(40, 1), (229, 1), (253, 1), (319, 2), (330, ...","[0, 0, 0, 0, 0, 0.153425917935, 0, 0, 0.341364..."
309fdbfc-227b-4588-9264-f0f4e3cadfcb,1994,"[comprehens, syntax, syntax, comprehens, close...","[(90, 3), (123, 1), (186, 1), (189, 5), (196, ...","[0, 0.298147649843, 0, 0, 0, 0, 0, 0, 0, 0.181..."
d5814bab-5fc2-4c31-92b7-543c7ce75cb4,2012,"[evalu, speaker, verif, secur, detect, hmmbase...","[(18, 1), (28, 1), (29, 2), (30, 1), (63, 1), ...","[0, 0, 0, 0, 0.132735923657, 0, 0, 0, 0.122869..."


## Clustering

# Topic Distribution of PoInf

In [52]:
# Create a new pandas table that merge the lookup_poinf and lookup_pub
# each pub in lookup_pub have a collab_id that have a list of collaborators by id
# We can ignore those that are not in the list of id for PoInf

# Create the list of ids for easy checking:
poinf_id = set(lookup_poinf.index)

# we can now create such an index:
pub_mapping = {str(_id):set() for _id in list(poinf_id)}

for row in lookup_pub.iterrows():
    pub_id = row[0]
    collab_ids = row[1]['collab_id']
    for _id in collab_ids:
        if _id in poinf_id:
            pub_mapping[_id].add(pub_id)

In [85]:
row_list = [{'id':k, 'pub_ids':v} for (k,v) in pub_mapping.items()]

In [86]:
# Add these pub_ids to the pandas df:
df_pubmapping = pd.DataFrame(row_list)

In [87]:
lookup_poinf_more = lookup_poinf.join(df_pubmapping.set_index('id'))

In [88]:
lookup_poinf_more.head(4)

,last_name,first_name,perseonal_url,position,parent,institute,full_name,institute_class,alias,pub_ids
id,,,,,,,,,,
003ec9bb-18aa-4e6e-95e9-359f0968262a,gray,gavin,http://www.research.ed.ac.uk/portal/en/persons...,research assistant,school of informatics,institute for computing systems architecture,gray gavin,5,NaN,{}
010f9bf0-c04c-4cfb-ab3d-ca150de1e706,jackson,paul,http://www.research.ed.ac.uk/portal/en/persons...,senior lecturer,school of informatics,institute for computing systems architecture,jackson paul,5,"jackson, p. b.|jackson, p.","{c5754b06-fcf9-4362-aa3a-1142589b5402, 167c4b6..."
02c86de2-0fc9-4f6d-aee9-93b0f7557c84,franke,bjoern,http://www.research.ed.ac.uk/portal/en/persons...,reader,school of informatics,institute of language cognition and computation,franke bjoern,2,"franke, b.","{9a3368cc-e69d-4ecf-bad1-b43ab0ac89a8, ab3fccd..."
02df029e-077b-407c-8e24-1392a9ed8060,sevilla-lara,laura,http://www.research.ed.ac.uk/portal/en/persons...,lecturer reader in image and vision computing,school of informatics,UNKNOWN,sevilla-lara laura,0,NaN,{}


In [ ]:
def getToks(pub_ids):
    out = []
    if len(pub_ids):
        for pub_id in pub_ids:
            out.extend(pub_toks[pub_toks.index == pub_id].summary_toks)
    return out

In [ ]:
lookup_poinf_more['toks'] = lookup_poinf_more.apply(lambda row: getToks(row.pub_ids), axis=1)